In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

#Restart kernel before running imports

import hsrb_interface
import rospy
import sys
from hsrb_interface import geometry
import cv2
from cv_bridge import CvBridge, CvBridgeError
import matplotlib.pyplot as plt
from sensor_msgs.msg import Image
import HF #helper functions/classes
import os
import csv
import numpy as np
 


#setting root directory to the parent directory of the catkin workspace 
path = os.getcwd()
ROOT_DIR = os.path.abspath(os.path.join(path, os.pardir))
os.chdir(ROOT_DIR)
print(ROOT_DIR)


/home/orlandothefraser/robot-grasp-detection_git


In [2]:
#ititializing HSR python API
robot = hsrb_interface.Robot()
omni_base = robot.get('omni_base')
whole_body = robot.get('whole_body')
gripper = robot.get('gripper')
tts = robot.get('default_tts')

In [6]:
#Setting the initial position of the arm and hand
whole_body.move_to_go()
whole_body.move_to_joint_positions({'arm_lift_joint': 0.4, 'arm_flex_joint': -1.57, 'arm_roll_joint': 0, 'wrist_roll_joint': -1.57})
gripper.command(1.2)

In [4]:
class HandCameraImage(object):
    def __init__(self):
        
        topic_name = "/hsrb/hand_camera/image_raw"
        self._bridge = CvBridge()
        self._rgb_image = None
        self._depth_image = None

        # Subscribe to hand camera topic
        self._image_sub = rospy.Subscriber(
            topic_name, Image, self.Image_CallBack)
        # Wait until connection
        rospy.wait_for_message(topic_name, Image, timeout=5.0)
        
    def Image_CallBack(self, data):
        try:
            self._rgb_image = self._bridge.imgmsg_to_cv2(data, "bgr8")
        except CvBridgeError as cv_bridge_exception:
            rospy.logerr(cv_bridge_exception)

In [7]:
import time
def main():
    
    robot = hsrb_interface.Robot()
    omni_base = robot.get('omni_base')
    whole_body = robot.get('whole_body')
    gripper = robot.get('gripper')
    tts = robot.get('default_tts')

    
    #rospy.init_node('hand_camera')
    try:
        hand_camera = HandCameraImage()
        spin_rate = rospy.Rate(30)

        rgb_image = hand_camera._rgb_image
        plt.imsave('./MiDaS/input/rgb.png',rgb_image)
        #cv2.imshow("Hand Camera Image", rgb_image)
            
        #Produce depth from rgb image
        t = time.time()
        ! python ./MiDaS/run.py
        e = time.time()
        print(e-t)
        #Produce grasp from depth
        #os.chdir('/home/orlandothefraser/catkin_ws')
        t = time.time()
        ! python ./ggcnn/Depth2Grasp.py --network ./ggcnn/ggcnn_weights_cornell/ggcnn_epoch_23_cornell --input_depth_path ./MiDaS/output/rgb.png --input_rgb_path ./MiDaS/input/rgb.png --vis 
        e = time.time()
        print(e-t)
        g = HF.get_grasp('./GraspOutput.csv')
            
        pos_error = g[0:3]
            
        step_vector = pos_error
    
        distance = (step_vector[0]**2+step_vector[1]**2)**0.5
        direction = (1/distance)*step_vector
            
        whole_body.move_end_effector_by_line((direction[1], direction[0], 0), distance)
            
        current_ang = 0
        ang_change = g[3]
        new_ang = current_ang - ang_change
        whole_body.move_to_joint_positions({'wrist_roll_joint': new_ang})
            
            
        rgb_image = hand_camera._rgb_image
        plt.imsave('./MiDaS/input/rgb.png',rgb_image)
        #cv2.imshow("Hand Camera Image", rgb_image)
            
        #Produce depth from rgb image
        ! python ./MiDaS/run.py
        #Produce grasp from depth
        ! python ./ggcnn/Depth2Grasp.py --network ./ggcnn/ggcnn_weights_cornell/ggcnn_epoch_23_cornell --input_depth_path ./MiDaS/output/rgb.png --input_rgb_path ./MiDaS/input/rgb.png --vis
            
        g = HF.get_grasp('./GraspOutput.csv')
            
        pos_error = g[0:3]
        
        #we dont want to go the entire way as there needs to be space for the object. 0.7 worked fairly wel
        step_vector = 0.7*pos_error
    
        distance = (step_vector[0]**2+step_vector[1]**2+step_vector[2]**2)**0.5
        direction = (1/distance)*step_vector
            
        whole_body.move_end_effector_by_line((direction[1], direction[0], direction[2]), distance)
            
        #width - add 1mm each side
        #gripper_width= g[4]+0.01
        #gripper.set_distance(gripper_width)
            
        gripper.apply_force(1.0)
            
        whole_body.move_to_joint_positions({'arm_lift_joint': 0.5})
            
        #cv2.waitKey(3)
        #spin_rate.sleep()
        
        print('test complete!')
            

    except rospy.ROSException as wait_for_msg_exception:
        rospy.logerr(wait_for_msg_exception)

    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()
    


initialize
device: cpu
Loading weights:  ./MiDaS/model-f46da743.pt
Using cache found in /home/orlandothefraser/.cache/torch/hub/facebookresearch_WSL-Images_master
start processing
  processing ./MiDaS/input/rgb.png (1/1)
finished
14.0364160538
/home/orlandothefraser/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/orlandothefraser/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  w

In [9]:
! pwd

/home/orlandothefraser/robot-grasp-detection_git
